In [101]:
import pandas as pd
import multiprocessing as mp
import os
import requests, zipfile, io
import re
import datetime
import string
import time
import glob
from glob import iglob
from datetime import datetime
import newspaper
from newspaper import Article
import pickle
import csv
from time import ctime
from langdetect import detect
import threading
from events2 import FilterCountry
from events_pickle8 import read_trigger_config
from events_pickle8 import filter_stories_bypickle
def Mappingdict(DateFile,MappingLanguageList,news_list,Csv):
    #NumNews = [0 for i in range(len(TriggerList))]
    TriggerList = MappingLanguageList
    BOPRList = ['s1','s2','s3','s4','s5','m1','m2']
    GCTCList = ['m3','m4','m5','m6','m7','m8','m9','m10','m11','m12','m13','m14','m15','m16','m17','m18','m19','m20','m21','m22','m23','m24']
    GISOList = ['m25','m26','m27','m28','m29','m30','m31','m32','m33','m34','m35','m36','m37','m38','m39','m40','m41']
    if len(TriggerList)==0:
        print('No TriggerList has been given')
    else:
        print('Start Mapping')
        nn = 0
        trigger_dict_eng = read_trigger_config(TriggerList[0])
        trigger_dict_span = read_trigger_config(TriggerList[1])
        fsplit = 'arg'+DateFile.split('.')[0]+'eng.p'
        num_line = len(news_list)
        with open(fsplit,'rb') as PickleFiles:
            feng = pickle.loads(PickleFiles.read())
        filter_stories_bypickle(feng,trigger_dict_eng,trigger_dict_span,num_line)
        NumNewsAll = 0
        TotalToneAll = 0
        TotalToneAllEng = 0
        TotalToneGovEng = 0
        TotalToneCtyEng = 0
        NumNewsAllEng = 0
        NumNewsGovEng = 0
        NumNewsCtyEng = 0
        TotalToneAllSpan = 0
        TotalToneGovSpan = 0
        TotalToneGCTCSpan = 0
        TotalToneGISOSpan = 0
        NumNewsAllSpan = 0
        NumNewsGovSpan = 0
        NumNewsGCTCSpan = 0
        NumNewsGISOSpan = 0
        for i in range(num_line):
            #print(feng[i].get_text())
            TotalToneAll += feng[i].get_tone()
            NumNewsAll += 1
            #print(news_list[i].get_language())
            #f.split('/')[-1].split('.')[0]
            #taxo = news_list[i].get_taxonomy()[0][0]
            if feng[i].get_language() != []:
                print(feng[i].get_language())
                if feng[i].get_language()[0] == 'en':
                    print(feng[i].get_language()[0])
                    TotalToneAllEng += feng[i].get_tone()
                    NumNewsAllEng += 1
                    if feng[i].get_taxonomy() !=[]:
                        taxo = feng[i].get_taxonomy()[-1][0]
                        print(feng[i].get_taxonomy())
                        if taxo in BOPRList:
                            TotalToneGovEng += feng[i].get_tone()
                            NumNewsGovEng += 1
                        if taxo in GCTCList:
                            TotalToneGovEng += feng[i].get_tone()
                            NumNewsGovEng += 1
                        if taxo in GISOList:
                            TotalToneGovEng += feng[i].get_tone()
                            NumNewsGovEng += 1
                if feng[i].get_language()[0] == 'es':
                    print(feng[i].get_language()[0])
                    TotalToneAllSpan += feng[i].get_tone()
                    NumNewsAllSpan += 1
                    if feng[i].get_taxonomy() !=[]:
                        taxo = feng[i].get_taxonomy()[-1][0].split('-')[0]
                        print(feng[i].get_taxonomy())
                        if taxo in BOPRList:
                            TotalToneGovSpan += feng[i].get_tone()
                            NumNewsGovSpan += 1
                        if taxo in GCTCList:
                            TotalToneGovSpan += feng[i].get_tone()
                            NumNewsGovSpan += 1
                        if taxo in GISOList:
                            TotalToneGovSpan += feng[i].get_tone()
                            NumNewsGovSpan += 1
                        
            if i < num_line - 1:
                if feng[i].get_sqldate() != feng[i+1].get_sqldate():
                    #print(news_list[i].get_sqldate())
                    Csv[nn][0] = feng[i].get_sqldate()
                    Csv[nn][1] = TotalToneAll
                    Csv[nn][2] = NumNewsAll
                    if NumNewsAll != 0:
                        Csv[nn][3] = TotalToneAll/NumNewsAll
                    Csv[nn][4] = TotalToneAllEng
                    Csv[nn][5] = NumNewsAllEng
                    if NumNewsAllEng != 0:
                        Csv[nn][6]  = TotalToneAllEng/NumNewsAllEng
                    Csv[nn][7] = TotalToneGovEng
                    Csv[nn][8] = NumNewsGovEng
                    if NumNewsGovEng != 0:
                        Csv[nn][9]  = TotalToneGovEng/NumNewsGovEng
                    Csv[nn][10] = TotalToneAllSpan
                    Csv[nn][11] = NumNewsAllSpan
                    if NumNewsAllSpan != 0:
                        Csv[nn][12]  = TotalToneAllSpan/NumNewsAllSpan
                    Csv[nn][13] = TotalToneGovSpan
                    Csv[nn][14] = NumNewsGovSpan
                    if NumNewsGovSpan != 0:
                        Csv[nn][15]  = TotalToneGovSpan/NumNewsGovSpan
                    NumNewsAll = 0
                    TotalToneAll = 0
                    TotalToneAllEng = 0
                    TotalToneGovEng = 0
                    TotalToneCtyEng = 0
                    TotalToneGISOEng = 0
                    NumNewsAllEng = 0
                    NumNewsGovEng = 0
                    NumNewsCtyEng = 0
                    NumNewsGISOEng = 0
                    TotalToneAllSpan = 0
                    TotalToneGovSpan = 0
                    TotalToneGCTCSpan = 0
                    TotalToneGISOSpan = 0
                    NumNewsAllSpan = 0
                    NumNewsGovSpan = 0
                    NumNewsGCTCSpan = 0
                    NumNewsGISOSpan = 0
                    nn += 1
                    #print(nn)
                    #print(ctime())
            if i == num_line - 1:
                Csv[nn][0] = feng[i].get_sqldate()
                Csv[nn][1] = TotalToneAll
                Csv[nn][2] = NumNewsAll
                if NumNewsAll != 0:
                    Csv[nn][3] = TotalToneAll/NumNewsAll
                Csv[nn][4] = TotalToneAllEng
                Csv[nn][5] = NumNewsAllEng
                if NumNewsAllEng != 0:
                    Csv[nn][6]  = TotalToneAllEng/NumNewsAllEng
                Csv[nn][7] = TotalToneGovEng
                Csv[nn][8] = NumNewsGovEng
                if NumNewsGovEng != 0:
                    Csv[nn][9]  = TotalToneGovEng/NumNewsGovEng
                Csv[nn][10] = TotalToneAllSpan
                Csv[nn][11] = NumNewsAllSpan
                if NumNewsAllSpan != 0:
                    Csv[nn][12]  = TotalToneAllSpan/NumNewsAllSpan
                Csv[nn][13] = TotalToneGovSpan
                Csv[nn][14] = NumNewsGovSpan
                if NumNewsGovSpan != 0:
                    Csv[nn][15]  = TotalToneGovSpan/NumNewsGovSpan
    return Csv
              
def WritetoCsv(DateFile,Csv):
    resultfile = 'resultfile'+DateFile.split('.')[0]+'gov.csv'
    with open(resultfile,'w', newline = '') as InputF:
        csv_write = csv.writer(InputF)
        csv_head1 = ['Date','TotalToneAll','NumNewsAll','AvgToneAll',
                     'TotalToneAllEng','NumNewsAllEng','AvgToneAllEng',
                     'TotalToneGovEng','NumNewsGovEng','AvgToneGovEng',
                     'TotalToneAllSpan','NumNewsAllSpan','AvgToneAllSpan',
                     'TotalToneGovSpan','NumNewsGovSpan','AvgToneGovSpan',]
        csv_write.writerow(csv_head1)
        csv_write.writerows(Csv)
    print('writing finished')

def events_tonestatistic(DateFile, ActorCountiesList, MappingLanguageList):
    """
    
    """
    #fileName = ['20150820.csv','20150830.csv','20150914.csv','20151009.csv','20151125.csv','20151221.csv']
    #ActorCountiesList = ['ARG','Covering']
    #MappingLanguageList = ['triggerlist-english-co.txt','triggerlist-Spanish-extend-f.txt']
    #DateFile = fileName[i]
    news_list = FilterCountry(ActorCountiesList,DateFile)
    Csv = [[0 for i in range(16)] for i in range(31)]
    print(ctime())
    Mappingdict(DateFile,MappingLanguageList,news_list,Csv)#mapping the news with the dictionary
    print(ctime())
    WritetoCsv(DateFile,Csv)

In [102]:
events_tonestatistic('20150826.csv', ['ARG','Covering'], ['triggerlist-english.txt','triggerlist-Spanish-extend-f.txt'])
print(ctime()+'end')


There are 99 items in News.
Wed May  1 16:08:16 2019
Start Mapping
['s1+TEXT+Currency conversion', 's2+TEXT+Currency inconvertibility', 's3+TEXT+Currency regulation', 's4+TEXT+Exchange inconvertibility', 'm1+AND+Government+Foreign+inconvertibility', 'm2+AND+Government+Foreign+export tax', 'm3+AND+Government+Foreign+Discriminatory regulation', 'm4+AND+Government+Foreign+Breach of contract', 'm5+AND+Government+Foreign+Reneging+contract', 'm6+AND+Government+Foreign+Reneging+agreement', 'm7+AND+Government+Foreign+Renege+contract', 'm8+AND+Government+Foreign+Renege+agreement', 'm9+AND+Government+Foreign+abrogate+contract', 'm10+AND+Government+Foreign+abrogate+agreement', 'm11+AND+Government+Foreign+abrogating+contract', 'm12+AND+Government+Foreign+abrogating+agreement', 'm13+AND+Government+Foreign+Renegotiating terms', 'm14+AND+Government+Foreign+tax payments+dispute', 'm15+AND+Government+Foreign+coersion+contract', 'm16+AND+Government+Foreign+coersion+agreement', 'm17+AND+Government+Forei

[]
5
[]
6
[]
7
[]
8
[]
9
[]
10
[]
11
[]
12
[]
13
[]
14
[]
15
[]
16
[]
17
[]
18
[]
19
[]
20
[]
21
[]
22
[]
23
[]
24
[]
25
[]
26
[]
27
[]
28
[]
29
[]
30
[]
31
[]
32
[]
33
[]
34
[]
35
[]
36
[]
37
[]
38
[]
39
[]
40
[]
41
[]
42
[]
43
[]
44
[]
45
[]
46
[]
47
[]
48
[]
49
[]
50
[]
51
[]
52
[]
53
[]
54
[]
55
[]
56
[]
57
[]
58
[]
59
[]
60
[]
61
[]
62
[]
63
[]
64
[]
65
[]
66
[]
67
[]
68
[]
69
[]
70
[]
71
[]
72
[]
73
[]
74
[]
75
[]
76
[]
77
[]
78
[]
79
[]
80
[]
81
[]
82
[]
83
[]
84
[]
85
[]
86
[]
87
[('m37-4', 'estado+extranjero+resto')]
88
[]
89
[]
90
[]
91
[]
92
[]
93
[]
94
[]
95
[]
96
[]
97
[]
98
[]
['en']
en
['es']
es
['en']
en
['es']
es
['en']
en
['en']
en
['en']
en
['en']
en
['en']
en
['en']
en
['en']
en
['es']
es
['es']
es
['es']
es
['es']
es
['es']
es
['en']
en
['en']
en
['en']
en
['es']
es
['es']
es
['es']
es
['es']
es
['es']
es
['es']
es
['en']
en
['en']
en
['en']
en
['en']
en
['en']
en
['es']
es
['es']
es
['es']
es
['en']
en
['en']
en
['es']
es
['es']
es
['es']
es
['es']
es
['en']
en
['

In [103]:
list1 = ['adgdg','123']
b = ['345']
list1.append(b)
print(list1[-1])

['345']
